In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, Label, Entry, Button
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import locale

# Configuración regional
locale.setlocale(locale.LC_ALL, 'en_US')

# Función para procesar el archivo de Excel y crear nueva hoja con DataFrame procesado
def process_excel_file(file_path, sala):
    df = pd.read_excel(file_path, sheet_name='Nuevo Sol Peruano', header=7)
    df = df.drop(df.columns[0], axis=1)
    columnas_especificas = ['Fecha Final', 'Cod. Máquina', 'Juego', 'Nro. Serie', 'Coinin', 'Win', 'Avbet x Game', 'Games Played']
    df = df[columnas_especificas]
    df['Cod. Máquina'] = pd.to_numeric(df['Cod. Máquina'], errors='ignore')
    df['Nro. Serie'] = df['Nro. Serie'].astype(str).str.lstrip('0')
    df['Fecha Final'] = pd.to_datetime(df['Fecha Final']).dt.strftime('%d/%m/%Y')
    df.insert(1, 'sala', sala)
    df.rename(columns={'Cod. Máquina': 'cod_maquina', 'Juego': 'juego', 'Nro. Serie': 'serie', 'Coinin': 'coin_in', 'Win': 'net_win', 'Avbet x Game': 'average_bet', 'Games Played': 'game_played'}, inplace=True)
    df = df.iloc[:-1]
    # Aplicar condiciones adicionales a las columnas average_bet y game_played
    df.loc[df['average_bet'] < 0, 'average_bet'] = 0
    df.loc[df['game_played'] < 0, 'game_played'] = 0
    df.loc[df['average_bet'] == df['coin_in'], 'average_bet'] = 0
    df.loc[df['game_played'] == 0, 'average_bet'] = 0
    df.loc[(df['game_played'] > 310000), ['game_played', 'average_bet']] = 0
    df = df.sort_values(['coin_in', 'game_played'], ascending=[True, True])
    # Renombrar columnas
    nuevos_nombres = {
        'Fecha Final': 'fecha',
        'Cod. Máquina': 'cod_maquina',
        'Juego': 'juego',
        'Nro. Serie': 'serie',
        'Coinin': 'coin_in',
        'Win': 'net_win',
        'Avbet x Game': 'average_bet'
    }
    df.rename(columns=nuevos_nombres, inplace=True)

    # Crear una nueva hoja en el archivo de Excel y escribir el DataFrame procesado
    archivo_excel = load_workbook(file_path)
    hoja_nueva = archivo_excel.create_sheet('Hoja Procesada')
    hoja_activa = archivo_excel.active
    for fila in dataframe_to_rows(df, index=False, header=True):
        hoja_nueva.append(fila)
    fuente = Font(name='Arial', size=9)
    relleno_encabezado = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    alineacion = Alignment(horizontal='left')
    for fila in hoja_nueva.iter_rows(min_row=1, max_row=1):
        for celda in fila:
            celda.font = fuente
            celda.fill = relleno_encabezado
            celda.alignment = alineacion

    # Guardar el archivo sobreescribiendo el original
    archivo_excel.save(file_path)

# Función para abrir el cuadro de diálogo de selección de archivo y obtener el valor de sala
def open_file_dialog():
    root = tk.Tk()
    root.title("Interfaz para Procesar Archivos")

    def process_file():
        sala_value = sala_entry.get()
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path and sala_value:
            process_excel_file(file_path, sala_value)

    Label(root, text="Ingrese ID de sala:").pack()
    sala_entry = Entry(root)
    sala_entry.pack()

    Button(root, text="Seleccionar Archivo", command=process_file).pack()

    root.mainloop()

# Llamar a la función para abrir la ventana de selección de archivo y valor de sala
open_file_dialog()
